In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
data  = pd.read_csv('Database/FINAL_EDGE_TABLE.csv', index_col = 0)
data = data.drop(columns = {'Unnamed: 0'}, axis = 1)
data.head()

,citypair,sum_departures_performed,sum_departures_scheduled,passengers,seats,avg_fuel_price,avg_stock_price,total_operating_expense,revenue,Aircraft_Unit_Cost ($ millions USD),...,dest_long,dest_city,Route_Distance,origin_population,origin_density,dest_population,dest_density,DEP_DELAY_NEW,ARR_DELAY_NEW,CANCELLED
0,ABQ-BWI,108.0,110.0,13736.0,15444.0,57.427544,52.842995,9585665.0,11058,74.0,...,-76.668297,Baltimore,1670.237770,765693.0,1155.5,2205092.0,2872.8,8.644068,9.440678,0.063492
1,ABQ-DAL,488.0,504.0,56344.0,74104.0,56.594778,53.008226,14229909.0,16207,74.0,...,-96.851799,Dallas,580.226586,765693.0,1155.5,5668165.0,1522.2,9.102740,8.075862,0.006803
2,ABQ-DEN,330.0,337.0,37782.0,47510.0,56.594778,53.008226,14229909.0,16207,74.0,...,-104.672996,Denver,349.103284,765693.0,1155.5,2650725.0,1805.7,18.624521,18.896154,0.003817
3,ABQ-HOU,326.0,335.0,39590.0,50650.0,56.927884,52.942134,23815574.0,27265,74.0,...,-95.278900,Houston,759.153724,765693.0,1155.5,5650910.0,1394.6,4.595506,4.044944,0.000000
4,ABQ-LAS,324.0,333.0,35966.0,46332.0,57.427544,52.842995,9585665.0,11058,74.0,...,-115.151817,Las Vegas,486.424110,765693.0,1155.5,2150373.0,1754.7,15.618321,13.946565,0.015038


In [3]:
# Utilization: boaded passengers concerning all departed flights’ available seats
data = data.assign(Utilization = data['passengers'] / data['seats'])
# Regularity: percentage of operated flights concerning planned flights
data = data.assign(Regularity = data['sum_departures_performed'] / data['sum_departures_scheduled'])
# Available seat miles (ASM): airline’s carrying capacity
data = data.assign(ASM = data['seats'] / data['Route_Distance'])
# Revenue Per Available Seat Mile (RASM): show financial performance
data = data.assign(RASM = data['revenue'] / data['ASM'])
data.head()

,citypair,sum_departures_performed,sum_departures_scheduled,passengers,seats,avg_fuel_price,avg_stock_price,total_operating_expense,revenue,Aircraft_Unit_Cost ($ millions USD),...,origin_density,dest_population,dest_density,DEP_DELAY_NEW,ARR_DELAY_NEW,CANCELLED,Utilization,Regularity,ASM,RASM
0,ABQ-BWI,108.0,110.0,13736.0,15444.0,57.427544,52.842995,9585665.0,11058,74.0,...,1155.5,2205092.0,2872.8,8.644068,9.440678,0.063492,0.889407,0.981818,9.246588,1195.900626
1,ABQ-DAL,488.0,504.0,56344.0,74104.0,56.594778,53.008226,14229909.0,16207,74.0,...,1155.5,5668165.0,1522.2,9.102740,8.075862,0.006803,0.760337,0.968254,127.715623,126.899118
2,ABQ-DEN,330.0,337.0,37782.0,47510.0,56.594778,53.008226,14229909.0,16207,74.0,...,1155.5,2650725.0,1805.7,18.624521,18.896154,0.003817,0.795243,0.979228,136.091530,119.088969
3,ABQ-HOU,326.0,335.0,39590.0,50650.0,56.927884,52.942134,23815574.0,27265,74.0,...,1155.5,5650910.0,1394.6,4.595506,4.044944,0.000000,0.781639,0.973134,66.719030,408.654024
4,ABQ-LAS,324.0,333.0,35966.0,46332.0,57.427544,52.842995,9585665.0,11058,74.0,...,1155.5,2150373.0,1754.7,15.618321,13.946565,0.015038,0.776267,0.972973,95.250213,116.094229


### Use the internel attributes extracted from the structure of airline network to detect 'anomaly' airports
- Degree centrality:  the number of connections a node has
- Betweenness centrality: how often a node appear on the shortest path


In [12]:
# Create a graph
G = nx.from_pandas_edgelist(data, source = 'Origin_Airport_Code', target = 'Destination_Airport_Code', create_using = nx.DiGraph, edge_attr = ['Utilization', 'Regularity', 'CANCELLED', 'ASM', 'RASM'])
# Calculate degree centrality for each airport
degree_centrality = nx.degree_centrality(G)
# Calculate betweenness centrality for each airport
betweenness_centrality = nx.betweenness_centrality(G)

In [13]:
# Get the nodes of the graph
nodes = [node for node in G.nodes()]
# Get the edges of the graph
edges = [edge for edge in G.edges()]

### Local Outlier Factor: identifies an outlier considering the density of its neighborhood
Only use this method here for detecting anomaly airports (sanity check)

#### Use degree centrality as the attribute

In [8]:
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt

# Compute LOF scores
X = np.array(list(degree_centrality.values())).reshape(-1, 1)
lof = LocalOutlierFactor(n_neighbors = 20)
lof_scores = lof.fit_predict(X)
outliers = [nodes[i] for i in np.where(lof.negative_outlier_factor_ < lof.offset_)[0]]
outliers

['BWI',
 'DAL',
 'DEN',
 'HOU',
 'LAS',
 'LAX',
 'MDW',
 'OAK',
 'PHX',
 'MCO',
 'TPA',
 'BNA',
 'CMH',
 'DCA',
 'IND',
 'MKE',
 'MSY',
 'SAN',
 'SAT',
 'STL',
 'SJC',
 'SMF']

#### Use betweenness centrality as the attribute

In [9]:
X = np.array(list(betweenness_centrality.values())).reshape(-1, 1)
lof = LocalOutlierFactor(n_neighbors = 20)
lof_scores = lof.fit_predict(X)
outliers = [nodes[i] for i in np.where(lof.negative_outlier_factor_ < lof.offset_)[0]]
outliers

['ABQ',
 'BWI',
 'DAL',
 'DEN',
 'HOU',
 'LAS',
 'LAX',
 'MDW',
 'OAK',
 'PHX',
 'ALB',
 'FLL',
 'MCO',
 'TPA',
 'ATL',
 'AUS',
 'BNA',
 'BOS',
 'CLE',
 'CMH',
 'DCA',
 'IND',
 'LGA',
 'MCI',
 'MKE',
 'MSP',
 'MSY',
 'PBI',
 'PHL',
 'PIT',
 'RSW',
 'SAN',
 'SAT',
 'STL',
 'ELP',
 'EWR',
 'SFO',
 'SJC',
 'SMF',
 'BDL',
 'BHM',
 'RDU',
 'BOI',
 'GEG',
 'BUF',
 'BUR',
 'PDX',
 'SLC',
 'GRR',
 'ISP',
 'MEM',
 'MHT',
 'OKC',
 'PVD',
 'ROC',
 'SDF',
 'SJU',
 'OMA',
 'SEA',
 'SNA',
 'ONT',
 'RNO',
 'TUS',
 'LGB']

### Use LOF to identify anomaly edges based on Utilization, Regularity, CANCELLED, ASM, RASM metrics

In [17]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Normalize the features so that they are on the same scale
X = data.loc[:, ['Utilization', 'Regularity', 'CANCELLED', 'ASM', 'RASM']]
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Compute the LOF scores based on the metrics
lof = LocalOutlierFactor(n_neighbors = 10)
lof_scores = lof.fit_predict(X)
outliers = [edges[i] for i in np.where(lof.negative_outlier_factor_ < lof.offset_)[0]]
outliers

[('BWI', 'SMF'),
 ('DEN', 'GRR'),
 ('DEN', 'MCO'),
 ('HOU', 'SNA'),
 ('LAS', 'MKE'),
 ('LAS', 'ONT'),
 ('LAX', 'BNA'),
 ('LAX', 'SFO'),
 ('MDW', 'BWI'),
 ('PHX', 'CMH'),
 ('PHX', 'DEN'),
 ('PHX', 'ELP'),
 ('FLL', 'IND'),
 ('MCO', 'ATL'),
 ('MCO', 'AUS'),
 ('MCO', 'PHL'),
 ('BNA', 'CLE'),
 ('DCA', 'STL'),
 ('DTW', 'STL'),
 ('PHL', 'HOU'),
 ('PHL', 'MCO'),
 ('RSW', 'BNA'),
 ('STL', 'RSW'),
 ('ELP', 'HOU'),
 ('HRL', 'AUS'),
 ('HRL', 'HOU'),
 ('SJC', 'HOU'),
 ('SJC', 'PHX'),
 ('BDL', 'TPA'),
 ('PDX', 'STL')]

### Adjacent matrix (need to explore)
- How to combine the edge attributes effectively to construct the adjacent matrix?

In [20]:
from scipy.sparse import csr_matrix

# get the adjacency matrix
A = nx.adjacency_matrix(G)

# convert the matrix to a numpy array
A = A.toarray()

/var/folders/vl/gqvqngs51qxf53rbxv14c96w0000gn/T/ipykernel_83956/1304958644.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


### Node2Vector: learn low-dimensional representation of nodes in the Graph
- Biased 2-order Random walk
- p, q controls the tradeoff between DFS and BFS

In [21]:
from node2vec import Node2Vec
# Initialize a Node2Vec object with the weighted graph and the weight keys
node2vec = Node2Vec(G, dimensions = 64, walk_length = 30, num_walks = 100, p = 1, q = 1, weight_key= ('Utilization', 'Regularity', 'CANCELLED', 'ASM', 'RASM'))

# Train the node2vec model
model = node2vec.fit(window = 10)

Computing transition probabilities:   0%|          | 0/85 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:01<00:00, 82.15it/s]


In [22]:
# get the node embeddings
embeddings = np.zeros((G.number_of_nodes(), model.vector_size))
for i, node in enumerate(G.nodes()):
    embeddings[i] = model.wv[node]

In [23]:
# Normalize the node embeddings
scaler = StandardScaler()
embeddings_normalized = scaler.fit_transform(embeddings)

### Local Outlier Factor (LOF)

In [24]:
lof = LocalOutlierFactor(n_neighbors = 30, contamination = 0.20)
lof.fit_predict(embeddings_normalized)
outliers = [nodes[i] for i in np.where(lof.negative_outlier_factor_ < lof.offset_)[0]]
outliers

['AMA',
 'ATL',
 'AUS',
 'GSP',
 'PBI',
 'RIC',
 'RSW',
 'HRL',
 'LBB',
 'BOI',
 'GEG',
 'ISP',
 'PWM',
 'ROC',
 'CRP',
 'ICT',
 'LGB']

### IsolationRandomForest
- emsenble of decision tree
- assumption: outliers are easier to be isolated
- anomaly score is based on the average path lengths between root and each node

In [26]:
from sklearn.ensemble import IsolationForest

# create an IsolationForest object
isolation_forest = IsolationForest(n_estimators = 100, contamination = 0.1, random_state = 42)

# fit the IsolationForest model to the embeddings
isolation_forest.fit(embeddings)
# get the average anomaly score
node_scores = isolation_forest.decision_function(embeddings)

In [27]:
# Identify the anomaly nodes based on the rank of anomaly score
k = 15  # number of anomalies to identify
anomaly_indices = (-node_scores).argsort()[:k]
anomaly_nodes = [nodes[i] for i in anomaly_indices]
print("Anomaly nodes:", anomaly_nodes)

Anomaly nodes: ['DTW', 'MSY', 'RDU', 'CMH', 'PIT', 'SAT', 'IND', 'ABQ', 'SDF', 'TUL', 'MSP', 'BOS', 'SEA', 'SLC', 'JAX']


In [28]:
# Or only pick nodes with average anomaly score below 0
outliers = [nodes[i] for i in np.where(node_scores < 0)[0]]
outliers

['GSP', 'PBI', 'RIC', 'HRL', 'BOI', 'GEG', 'ISP', 'CRP', 'LGB']

### One class SVM 
Since we don't have the labels for the airport such that we can assume all nodes belongs to only one class. OC-SVM uses the origin as the other class

In [30]:
from sklearn import svm

# Train the One-Class SVM
svm = svm.OneClassSVM(nu = 0.01, kernel = "rbf", gamma = 0.1)
svm.fit(embeddings_normalized)

# Predict the anomaly scores for all embeddings
anomaly_scores = svm.decision_function(embeddings_normalized)

# Identify nodes as anomalies if its anomaly scoce is below zero
anomaly_nodes = [nodes[i] for i in np.where(anomaly_scores < 0)[0]]
print("Anomaly nodes:", anomaly_nodes)


Anomaly nodes: ['BWI', 'DAL', 'DEN', 'HOU', 'LAS', 'LAX', 'OAK', 'PHX', 'ALB', 'FLL', 'MCO', 'TPA', 'AUS', 'BNA', 'BOS', 'CLE', 'GSP', 'IAD', 'JAX', 'LGA', 'MCI', 'PBI', 'SAN', 'ELP', 'HRL', 'SFO', 'SJC', 'BDL', 'BHM', 'PNS', 'BOI', 'BUR', 'PDX', 'SLC', 'CHS', 'CLT', 'ECP', 'ISP', 'MHT', 'OKC', 'ORF', 'PVD', 'SJU', 'CRP', 'LIT', 'OMA', 'SEA', 'SNA', 'TUL', 'RNO', 'TUS', 'DSM', 'LGB']


In [31]:
# Or we can set the threshold based on the distribution of the anomaly scores for all nodes
scores = svm.score_samples(embeddings_normalized)
thresh = np.quantile(scores, 0.10)
anomaly_nodes = [nodes[i] for i in np.where(scores <= thresh)[0]]
print("Anomaly nodes:", anomaly_nodes)

Anomaly nodes: ['OAK', 'BOS', 'GSP', 'ELP', 'SFO', 'CHS', 'ECP', 'PVD', 'OMA']


### K means

In [22]:
from sklearn.cluster import KMeans

In [23]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(embeddings_normalized)
kmeans.predict(embeddings_normalized)

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

#### Performing dimension reduction before implement K-means? Does 0 represent outlier?)

### Graph Convolutional Autoencoder (only edge with edge attributes information)
- encoder: graph convolutional layers with a fully connected layer
- decoder: mirrors the structure of encoders in a reverse order
- loss function: MSE
- anomaly nodes: Identify the nodes with the highest number of anomalous edges as abnormal nodes
- threshold choice